In [1]:
import pymongo
import pandas as pd
import jieba
import numpy as np
import string
from collections import Counter
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Embedding, Flatten
from keras.models import Sequential

In [72]:
# mongodb uri
conn_str = "mongodb://1.15.118.125:27017/"
# 客户端连接
client = pymongo.MongoClient(conn_str, serverSelectionTimeoutMS=5000)
collection_user = client.get_database("Lab").get_collection("Mission")

In [73]:
# 查询 title 与对应的 tags
title_with_tags = {}
for doc in collection_user.find():
    mission_tags = doc.get("missionTags")
    mission_title = doc.get("title")
    title_with_tags[mission_title] = mission_tags

# pands 格式化输出
ser_tag_tags = pd.Series(title_with_tags)
ser_tag_tags.head(len(ser_tag_tags))

学生骨干成长训练营举办暑期专题培训                     [学生工作, 培训会]
新生教师班主任工作研讨会召开                    [教学建设, 研讨会, 迎新]
2022级本科生线上新生家长会                    [师生交流, 会议, 迎新]
2022级研究生新生入学教育活动                       [迎新, 主题教育]
研究生班级新生见面会顺利举行                         [迎新, 班级活动]
                                       ...       
管理学院第二届工商管理学科高端论坛系列活动: 学科建设座谈会        [教学建设, 交流会]
新进老师培训                                [教学建设, 培训会]
学生骨干训练营的结营仪式                           [学生工作, 仪式]
十二月舍导交流                              [师生交流, 校园生活]
中国管理学前沿研究研讨会: 开幕式与主报告             [学术, 仪式, 汇报/报告]
Length: 101, dtype: object

In [78]:
# 忽略词
eng_words = list(string.ascii_lowercase + string.ascii_uppercase)
exclude_words = ['，', '“', '”', '、', '《', '》', '。', ':', '-', '的', '?', ' ', ',', '&','月','第二届', '第一次','日','与','华中科技大学','年度','顺利','—','电脑','暨','大学','关西','十一月','学院','来','｜', '第一','级','商学院','之','做','科','会','|','啦']
for w in eng_words:
    exclude_words.append(w)
for i in range(10):
    exclude_words.append(str(i))
print(exclude_words)

['，', '“', '”', '、', '《', '》', '。', ':', '-', '的', '?', ' ', ',', '&', '月', '第二届', '第一次', '日', '与', '华中科技大学', '年度', '顺利', '—', '电脑', '暨', '大学', '关西', '十一月', '学院', '来', '｜', '第一', '级', '商学院', '之', '做', '科', '会', '|', '啦', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [79]:
# 分割 title 的词语
total_words = []
title_with_words = {}
for title in title_with_tags.keys():
    for i in exclude_words:
        title = title.replace(i, '')
    title_with_words[title] = []
    # jieba拆分方法
    words = jieba.cut(title, cut_all=False)
    for word in words:
        total_words.append(word)
        title_with_words[title].append(word)

fre = Counter(total_words)
word_fre = sorted(fre.items(), key=lambda i: i[1], reverse=True)
print("总词数: " + len(word_fre).__str__())
print(word_fre)

总词数: 310
[('活动', 25), ('开展', 15), ('班', 14), ('分享', 13), ('举行', 12), ('管理', 11), ('主题', 10), ('党', 10), ('召开', 9), ('二十大', 9), ('精神', 9), ('新生', 8), ('宣讲', 7), ('论坛', 7), ('学习', 7), ('学生', 6), ('举办', 6), ('培训', 6), ('入学', 6), ('年', 6), ('大', 6), ('学术', 6), ('见面', 5), ('交流', 5), ('竞赛', 5), ('喻园', 5), ('党支部', 5), ('本', 5), ('知识', 5), ('骨干', 4), ('专题', 4), ('教育', 4), ('教授', 4), ('师生', 4), ('座谈', 4), ('十一', 4), ('党史', 4), ('德育', 4), ('训练营', 3), ('暑期', 3), ('工作', 3), ('研讨', 3), ('党员', 3), ('舍导进', 3), ('宿舍', 3), ('有限公司', 3), ('国奖', 3), ('成功', 3), ('杯', 3), ('院长', 3), ('生', 3), ('工商', 3), ('经验', 3), ('团支部', 3), ('团', 3), ('贯彻', 3), ('仪式', 3), ('项目', 3), ('集体', 3), ('报告', 3), ('领学团', 3), ('导师', 3), ('成长', 2), ('教师', 2), ('班主任', 2), ('实践', 2), ('心理', 2), ('简历', 2), ('秋季', 2), ('学期', 2), ('足球赛', 2), ('导航', 2), ('教学', 2), ('计', 2), ('专业课程', 2), ('本商', 2), ('副教授', 2), ('专业', 2), ('联合', 2), ('朋辈', 2), ('工商管理', 2), ('管理学', 2), ('我院', 2), ('议', 2), ('国际交流', 2), ('我为人人', 2), ('服务队', 2), ('义务', 2), ('维

In [80]:
for tuple in word_fre:
    if tuple[1] == 1:
        total_words.remove(tuple[0])
print(total_words.__len__())


433


In [81]:
print(set(total_words))
exclude_words = []

{'年', '精神', '党员', '知识', '宣讲', '高端', '举行', '计', '党史', '学术', '贯彻', '报告', '杯', '德育', '政策', '工商管理学', '足球赛', '生', '特色', '义务', '优秀', '工商', '专题', '宿舍', '学生', '二十大', '议', '本商', '入学', '新生', '秋季', '院长', '大', '教育', '交流', '项目', '国奖', '学长', '舍导进', '国际交流', '论坛', '座谈', '暑期', '实践', '导航', '教授', '我院', '开展', '有限公司', '举办', '选调', '备课', '活动', '成功', '见面', '师生', '导师', '召开', '本', '党支部', '仪式', '学子', '主题', '党', '十一', '分享', '班主任', '服务队', '工作', '学习', '副教授', '党课', '团支部', '研讨', '！', '系列', '简历', '喻园', '专业课程', '维修', '骨干', '领学团', '班', '教师', '管理学', '学期', '教学', '管理', '舍导', '心理', '我为人人', '训练营', '集体', '联合', '成长', '竞赛', '专业', '经验', '工商管理', '朋辈', '培训', '破冰', '团'}


In [82]:
# 查询 tag 的频次
def get_tag_times(dict_value_tags):
    tag_with_time = {}
    for key in dict_value_tags:
        for tag in dict_value_tags[key]:
            if tag not in tag_with_time:
                tag_with_time[tag] = 1
            else:
                tag_with_time[tag] += 1
    return tag_with_time


# 获取 tags 列表
tags_list = list(get_tag_times(title_with_tags).keys())
print(len(tags_list))
print(tags_list)

29
['学生工作', '培训会', '教学建设', '研讨会', '迎新', '师生交流', '会议', '主题教育', '班级活动', '就业发展', '校园生活', '党团活动', '社会实践', '调研活动', '交流会', '奖学金', '宣讲会', '其他活动', '文体活动', '赛事', '仪式', '学术', '讲座', '趣味活动', '答辩', '生活会', '采访', '课程学习', '汇报/报告']


In [83]:
# 构建词语数据矩阵
dist = np.zeros((len(title_with_tags), len(set(total_words))))
print(dist.shape)

(101, 103)


In [84]:
# 将数据填入矩阵
total_words_list = list(set(total_words))
title_list = list(title_with_words.keys())
for i in range(len(title_list)):
    print(title_with_words[title_list[i]])
    for word in title_with_words[title_list[i]]:
        for j in range(len(total_words_list)):
            if word == total_words_list[j]:
                print(word)
                dist[i][j] += 1

['学生', '骨干', '成长', '训练营', '举办', '暑期', '专题', '培训']
学生
骨干
成长
训练营
举办
暑期
专题
培训
['新生', '教师', '班主任', '工作', '研讨', '召开']
新生
教师
班主任
工作
研讨
召开
['本生线', '上', '新生', '家长']
新生
['研究生', '新生', '入学', '教育', '活动']
新生
入学
教育
活动
['研究生班', '新生', '见面', '举行']
新生
见面
举行
['就业', '动员', '年', '大']
年
大
['年', '红色', '领航员', '党员', '舍导进', '宿舍', '活动']
年
党员
舍导进
宿舍
活动
['赴', '湖北', '稳健', '医疗', '集团', '有限公司', '开展', '暑期', '实践', '调研', '活动']
有限公司
开展
暑期
实践
活动
['年', '暑期', '社', '实践', '分享', '交流']
年
暑期
实践
分享
交流
['年', '新生', '安全', '心理', '主题', '教育']
年
新生
心理
主题
教育
['国奖', '国励', '宣讲', '举办']
国奖
宣讲
举办
['简历', '指导', '对', '简历', '修改', '活动', '成功', '举办']
简历
简历
活动
成功
举办
['秋季', '学期', '教职工', '大', '举行']
秋季
学期
大
举行
['新生', '杯', '足球赛']
新生
杯
足球赛
['开学典礼', '入学', '导航']
入学
导航
['华中', '大', '年', '秋季', '学期', '教师', '教学', '竞赛', '赛前', '培训']
大
年
秋季
学期
教师
教学
竞赛
培训
['喻园', '管理', '论坛', '香港', '城市', '教授', '陈旸', '旸', '学术', '分享']
喻园
管理
论坛
教授
学术
分享
['计', '专业课程', '组', '研讨', '召开']
计
专业课程
研讨
召开
['本商', '院长', '矢田勝俊', '教授', '学术', '分享']
本商
院长
教授
学术
分享
['本商', '副教授', '李振', '学术', '分享']
本商
副教授


In [85]:
# 转化为 pd.DataFrame
data_words = pd.DataFrame(dist)
print(total_words_list)

['年', '精神', '党员', '知识', '宣讲', '高端', '举行', '计', '党史', '学术', '贯彻', '报告', '杯', '德育', '政策', '工商管理学', '足球赛', '生', '特色', '义务', '优秀', '工商', '专题', '宿舍', '学生', '二十大', '议', '本商', '入学', '新生', '秋季', '院长', '大', '教育', '交流', '项目', '国奖', '学长', '舍导进', '国际交流', '论坛', '座谈', '暑期', '实践', '导航', '教授', '我院', '开展', '有限公司', '举办', '选调', '备课', '活动', '成功', '见面', '师生', '导师', '召开', '本', '党支部', '仪式', '学子', '主题', '党', '十一', '分享', '班主任', '服务队', '工作', '学习', '副教授', '党课', '团支部', '研讨', '！', '系列', '简历', '喻园', '专业课程', '维修', '骨干', '领学团', '班', '教师', '管理学', '学期', '教学', '管理', '舍导', '心理', '我为人人', '训练营', '集体', '联合', '成长', '竞赛', '专业', '经验', '工商管理', '朋辈', '培训', '破冰', '团']


In [86]:
# tag 矩阵构建
# 定义 0 矩阵
dist_matrix = np.zeros((len(title_with_tags), len(get_tag_times(title_with_tags)),))
# 获取 tags 列表
tags_list = list(get_tag_times(title_with_tags).keys())
# 获取 title 的 list
title_list = list(title_with_tags.keys())
# 根据 tag 生成矩阵
for mission_index in range(len(title_with_tags)):
    for tag in title_with_tags[title_list[mission_index]]:
        for tag_index in range(len(tags_list)):
            if tag == tags_list[tag_index]:
                dist_matrix[mission_index][tag_index] = 1

data_label = pd.DataFrame(dist_matrix)

In [107]:
# 分割数据为训练集与测试集
X_train, X_test, y_train, y_test = train_test_split(data_words, data_label, test_size=0.5, random_state=0)

In [108]:
# 模型定义
def deep_model(feature_dim, label_dim):
    model = Sequential()
    print("create model. feature_dim ={}, label_dim ={}".format(feature_dim, label_dim))
    # model.add(Embedding(5, 32, input_length=79))
    # model.add(Flatten())
    model.add(Dense(1024, activation='relu', input_dim=feature_dim))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(label_dim, activation='softmax'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [109]:
# 模型训练函数
def train_deep(X_train, y_train, X_test, y_test, epochs):
    feature_dim = X_train.shape[1]
    label_dim = y_train.shape[1]
    model = deep_model(feature_dim, label_dim)
    model.summary()
    model.fit(X_train, y_train, batch_size=16, epochs=epochs, validation_data=(X_test, y_test))
    return model

In [110]:
# 开始训练
my_model = train_deep(X_train, y_train, X_test, y_test, 20)

create model. feature_dim =103, label_dim =29
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 1024)              106496    
                                                                 
 dense_15 (Dense)            (None, 128)               131200    
                                                                 
 dense_16 (Dense)            (None, 29)                3741      
                                                                 
Total params: 241,437
Trainable params: 241,437
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
4/4 [==============================] - 1s 74ms/step - loss: 0.6765 - accuracy: 0.0600 - val_loss: 0.6199 - val_accuracy: 0.0588
Epoch 2/20
4/4 [==============================] - 0s 14ms/step - loss: 0.5951 - accuracy: 0.1000 - val_loss: 0.5337 - val_accur

In [116]:
pre_y = my_model.predict(data_words)
print(type(pre_y))
print(pre_y.shape[0])
for i in range(pre_y.shape[0]):
    pre_tags = {}
    for j in range(pre_y[i].shape[0]):
        if pre_y[i][j] > 0.06:
            pre_tags[tags_list[j]] = pre_y[i][j]
    pre_tags_sort = sorted(pre_tags.items(),  key=lambda d: d[1], reverse=True)
    for key in pre_tags_sort:
        print(list(title_with_tags.keys())[i] +"  "+ key.__str__())


4/4 [==============================] - 0s 3ms/step
<class 'numpy.ndarray'>
101
学生骨干成长训练营举办暑期专题培训  ('培训会', 0.69595164)
学生骨干成长训练营举办暑期专题培训  ('学生工作', 0.11579029)
学生骨干成长训练营举办暑期专题培训  ('教学建设', 0.07416485)
学生骨干成长训练营举办暑期专题培训  ('会议', 0.04284234)
新生教师班主任工作研讨会召开  ('教学建设', 0.43382463)
新生教师班主任工作研讨会召开  ('师生交流', 0.103183016)
新生教师班主任工作研讨会召开  ('党团活动', 0.060478542)
新生教师班主任工作研讨会召开  ('迎新', 0.044092037)
新生教师班主任工作研讨会召开  ('宣讲会', 0.043618627)
新生教师班主任工作研讨会召开  ('会议', 0.041490406)
新生教师班主任工作研讨会召开  ('文体活动', 0.04078462)
新生教师班主任工作研讨会召开  ('校园生活', 0.03857714)
2022级本科生线上新生家长会  ('教学建设', 0.122395135)
2022级本科生线上新生家长会  ('文体活动', 0.06846909)
2022级本科生线上新生家长会  ('宣讲会', 0.0659275)
2022级本科生线上新生家长会  ('仪式', 0.065358475)
2022级本科生线上新生家长会  ('迎新', 0.064077325)
2022级本科生线上新生家长会  ('赛事', 0.06385316)
2022级本科生线上新生家长会  ('会议', 0.060399383)
2022级本科生线上新生家长会  ('其他活动', 0.041726273)
2022级本科生线上新生家长会  ('讲座', 0.041146904)
2022级本科生线上新生家长会  ('党团活动', 0.040064633)
2022级本科生线上新生家长会  ('师生交流', 0.039434083)
2022级本科生线上新生家长会  ('就业发展', 0.030965772)
2022级本科生线上新生家长会